# 지표 탐색 및 파이프라인 구축하기
---

### 목표
1. 지표 탐색(AARRR)
    - Acquisition(획득) : 회원가입 기준
    - Activation(활성화) : 회원가입 후 첫 투표
    - Retention(유지) : 다른 날 재접속하여 투표를 하는가
    - Revenue(수익) : 하트 구매
    - Referral(추천) : 1인당 초대한 친구 수

## 필요 라이브러리 정리
---
#### 1. 라이브러리 불러오기


In [3]:
## 1.필요 라이브러리 정리 
import pandas as pd

# object 형식으로 저장된 변수를 list 형식으로 바꾸기 위해서
import ast

# GCS 파일 경로에서 데이터 불러오기 위한 라이브러리
from google.cloud import storage
from google.oauth2 import service_account

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
import matplotlib.dates as mdates

### 2. 데이터 불러오기 실행 확인

In [4]:
## 2. GCS에서 데이터 불러오기
df = pd.read_parquet(
    "gs://my-advanced_data-bucket/votes/accounts_timelinereport.parquet",
    storage_options={"token": "/home/user/codeit_project/codeit-project-docker/config/key.json"}
)

display(df.head())

,id,reason,created_at,reported_user_id,user_id,user_question_record_id
0,28,타인을 사칭함,2023-05-06 04:44:57,874587,885082,3920588
1,37,친구를 비하하거나 조롱하는 어투,2023-05-06 05:41:19,881048,881298,4018679
2,46,친구를 비하하거나 조롱하는 어투,2023-05-06 06:26:59,887882,881945,4120914
3,64,허위 사실 언급,2023-05-06 07:07:46,888610,893684,4143049
4,65,허위 사실 언급,2023-05-06 07:07:52,888610,893684,4143049


## 데이터 가져오기
---
### 1. Votes 데이터 가져오기
### 2. Votes 데이터 내 전처리
### 3. Hackle 데이터 가져오기
### 4. Hackle 데이터 내 전처리

In [5]:
# 인증
key_path = "/home/user/codeit_project/codeit-project-docker/config/key.json"
credentials = service_account.Credentials.from_service_account_file(key_path)

# GCS 클라이언트 생성
client = storage.Client(credentials=credentials)
bucket = client.bucket("my-advanced_data-bucket")

# # 'votes/' 경로에 있는 .parquet 파일 리스트 가져오기
# vote_blobs = bucket.list_blobs(prefix="votes/")
# vote_file_list = [f"gs://my-advanced_data-bucket/{blob.name}" for blob in vote_blobs if blob.name.endswith(".parquet")]

# # 결과 확인
# print("Votes files in GCS:")
# print(vote_file_list)
# print()

# 'hackle/' 경로에 있는 .parquet 파일 리스트 가져오기
hackle_blobs = bucket.list_blobs(prefix="hackle/")
hackle_file_list = [f"gs://my-advanced_data-bucket/{blob.name}" for blob in hackle_blobs if blob.name.endswith(".parquet")]

# 결과 확인
print("Hackle files in GCS:")
print(hackle_file_list)


Hackle files in GCS:
['gs://my-advanced_data-bucket/hackle/device_properties.parquet', 'gs://my-advanced_data-bucket/hackle/hackle_events.parquet', 'gs://my-advanced_data-bucket/hackle/hackle_properties.parquet', 'gs://my-advanced_data-bucket/hackle/user_properties.parquet']


In [6]:
# 결과 저장용 딕셔너리
df_dict = {}

# # vote 내 각 파일을 불러와서 딕셔너리에 저장
# for title in vote_file_list:
#     gcs_path = title
#     df_file_name = title.split('/')[-1].replace('.parquet', '')
    
#     # 데이터 불러오기
#     df = pd.read_parquet(gcs_path, storage_options={"token": key_path})
    
#     # 딕셔너리에 저장
#     df_dict[df_file_name] = df

#     # 출력
#     print(f"{df_file_name} 데이터 프레임 확인")
#     print("="*50)
#     print(df.head())
#     print(f"{df_file_name} 데이터 프레임 크기 : {df.shape}")
#     print()
#     print(f"{df_file_name} 기본 정보 확인")
#     df.info()
#     print("\n" + "-"*70 + "\n")
    
# hackle 내 각 파일을 불러와서 딕셔너리에 저장
for title in hackle_file_list:
    gcs_path = title
    df_file_name = title.split('/')[-1].replace('.parquet', '')
    
    # 데이터 불러오기
    df = pd.read_parquet(gcs_path, storage_options={"token": key_path})
    
    # 딕셔너리에 저장
    df_dict[df_file_name] = df

    # 출력
    print(f"{df_file_name} 데이터 프레임 확인")
    print("="*50)
    print(df.head())
    print(f"{df_file_name} 데이터 프레임 크기 : {df.shape}")
    print()
    print(f"{df_file_name} 기본 정보 확인")
    df.info()
    print("\n" + "-"*70 + "\n")

device_properties 데이터 프레임 확인
   id                             device_id device_model device_vendor
0   1  000007C9-E103-4EB5-9777-A9084D4952DF   iPhone14,7         Apple
1   2  00002245-458F-4CDD-8533-B448CD43DBD2   iPhone14,7         Apple
2   3  00012620-313A-4502-9F8D-8DAB7443215B   iPhone14,5         Apple
3   4  000137bc-80de-4bb5-b61d-df7f217a4501     SM-F711N       samsung
4   5  000227D6-B782-4367-91C4-486B76DF9E37   iPhone12,3         Apple
device_properties 데이터 프레임 크기 : (252380, 4)

device_properties 기본 정보 확인
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252380 entries, 0 to 252379
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             252380 non-null  int64 
 1   device_id      252380 non-null  object
 2   device_model   252380 non-null  object
 3   device_vendor  252380 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.7+ MB

-------------------------------------------------

## Acquisition 
---
1. Acquistion 에 사용될 수 있는 데이터가 어떤 것들이 있는지 확인 
2. Acquistion 에 사용될 수 있는 데이터 가져오기
3. Acquisition 에 사용될 수 있는 데이터의 컬럼 확인


vote 내에서 이벤트(signup) 처럼 알 수 없어서 획득 알기 어려움
- vote 에서는 첫 출석 -> 회원가입한 날로 생각하게 된다면 어떻게 되는지 그래프 그려볼 수 있음음
- hackle event 데이터가 기간이 짧을거라 그렇다면 특정 시점에서 획득이 이 정도였다 등으로 진행 예정

In [7]:
display(df_dict["hackle_events"].head())

df_dict["hackle_events"].dtypes

,event_id,event_datetime,event_key,session_id,id,item_name,page_name,friend_count,votes_count,heart_balance,question_id
0,00000533-3f1c-4b3b-81f1-0c8f35754b4e,2023-07-18 19:40:17,$session_start,4OzYh3seq3VKytpSn5pvQkZNQii1,00000533-3f1c-4b3b-81f1-0c8f35754b4e,,,NaN,NaN,NaN,NaN
1,00000716-27e9-4e72-a602-d0ce61784b06,2023-07-18 21:07:24,click_question_open,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,00000716-27e9-4e72-a602-d0ce61784b06,,,64.0,436.0,4830.0,NaN
2,000007c8-68ce-40e6-9b1e-f0e34e8ff9cc,2023-08-06 20:18:03,click_bottom_navigation_profile,6bcea65d-9f40-46fc-888c-700fe707483f,000007c8-68ce-40e6-9b1e-f0e34e8ff9cc,,,26.0,174.0,4729.0,NaN
3,00000981-5e2a-4111-993e-4f1891ad9a53,2023-08-05 01:46:10,view_shop,XVYNT6zfhFWqIg9omwg2AHDjTLx2,00000981-5e2a-4111-993e-4f1891ad9a53,,,61.0,44.0,142.0,NaN
4,00000a7a-ba72-4332-b4a9-7910670aaeb2,2023-07-24 15:03:37,click_bottom_navigation_lab,XFB2SPiGfjbVhvJ3Q3DBsaT3m2B3,00000a7a-ba72-4332-b4a9-7910670aaeb2,,,119.0,545.0,3287.0,NaN


event_id                  object
event_datetime    datetime64[ms]
event_key                 object
session_id                object
id                        object
item_name                 object
page_name                 object
friend_count             float64
votes_count              float64
heart_balance            float64
question_id              float64
dtype: object

In [8]:
# df_dict["hackle_events"].session_id 와 df_dict["hackle_properties"].session_id

hackel_events_id_list = list(df_dict["hackle_events"].session_id.unique())
hackle_properties_id_list = list(df_dict["hackle_properties"].session_id.unique())

# hackle_events와 hackle_properties의 session_id가 일치하는지 확인
if set(hackel_events_id_list) == set(hackle_properties_id_list):
    print("hackle_events와 hackle_properties의 session_id가 일치합니다.")
else:
    print("hackle_events와 hackle_properties의 session_id가 일치하지 않습니다.")
    print(f"hackle_events id 개수: {len(hackel_events_id_list)}")
    print(f"hackle_properties id 개수: {len(hackle_properties_id_list)}")

hackle_events와 hackle_properties의 session_id가 일치합니다.


In [9]:
# hackle_properties의 user_id와 user_properties의 user_id가 일치하는지 확인
hackle_properties_user_id_list = list(df_dict["hackle_properties"].user_id.unique())
hackle_user_properties_user_id_list = list(df_dict["user_properties"].user_id.unique())
if set(hackle_properties_user_id_list) == set(hackle_user_properties_user_id_list):
    print("hackle_properties와 hackle_user_properties의 user_id가 일치합니다.")
else:
    print("hackle_properties와 hackle_user_properties의 user_id가 일치하지 않습니다.")
    print(f"hackle_properties user_id 개수: {len(hackle_properties_user_id_list)}")
    print(f"hackle_user_properties user_id 개수: {len(hackle_user_properties_user_id_list)}")

hackle_properties와 hackle_user_properties의 user_id가 일치하지 않습니다.
hackle_properties user_id 개수: 327381
hackle_user_properties user_id 개수: 230819


**합리적인 의심**       
- hackle_events와 hackle_properties를 합치기 위해서는 session_id로 병합 필요
    - why? 두 session 고유값이 같음
- hackle_properties와 user_properties를 합치기 위해서는 user_id로 병합 필요
    - user_id로 병합하는 이유는 무엇? hackle_properties는 어떤 이벤트 관련해서 알려주는 데이터(즉, 어플을 실행하고 회원가입을 하고 로그인을 하고 투표를 하고 등등)
    - 반면 user_properties는 회원가입을 완료한 회원들을 기준
    - 즉 유저가 적을 수 밖에 없음!!
        - 확인하려고 한다면? user_properties에 있는 user_id는 회원가입했던 기록이 있다던가?(데이터 기간이 짤려있어서 비율이 높아도 됨)

In [16]:
# 유저 아이디 별 session 개수 확인
user_session_counts = df_dict["hackle_properties"].groupby("user_id")["session_id"].nunique().reset_index()
user_session_counts.columns = ["user_id", "session_count"]
print("유저 아이디 별 세션 개수:")
print(user_session_counts.head())

유저 아이디 별 세션 개수:
                        user_id  session_count
0                                        78197
1  001zjRPEGWWVou4CbhLU7JQ9K9W2              1
2  002KHtytQcVvLzOzvprOv0sxVTM2              1
3  002r3iOqdkTogNK959c80mHZpIz2              1
4  008tC2rC0BaMMbCssEPu71R8kGm1              1


In [10]:
# hackle_events를 통한 hackle 데이터 기간 범위 확인
start_date = df_dict["hackle_events"].event_datetime.min()
end_date = df_dict["hackle_events"].event_datetime.max()
print(f"Hackle 데이터 기간: {start_date} ~ {end_date}")

Hackle 데이터 기간: 2023-07-18 00:00:00 ~ 2023-08-10 23:59:59


-> 우리가 집중적으로 보는 내용은 23.05월부터 24.05월(vote/accounts_attendance) 중 23.07 말 ~ 23.08 초 확인

In [11]:
display(df_dict["hackle_events"].head())
display(df_dict["hackle_properties"].head())

,event_id,event_datetime,event_key,session_id,id,item_name,page_name,friend_count,votes_count,heart_balance,question_id
0,00000533-3f1c-4b3b-81f1-0c8f35754b4e,2023-07-18 19:40:17,$session_start,4OzYh3seq3VKytpSn5pvQkZNQii1,00000533-3f1c-4b3b-81f1-0c8f35754b4e,,,NaN,NaN,NaN,NaN
1,00000716-27e9-4e72-a602-d0ce61784b06,2023-07-18 21:07:24,click_question_open,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,00000716-27e9-4e72-a602-d0ce61784b06,,,64.0,436.0,4830.0,NaN
2,000007c8-68ce-40e6-9b1e-f0e34e8ff9cc,2023-08-06 20:18:03,click_bottom_navigation_profile,6bcea65d-9f40-46fc-888c-700fe707483f,000007c8-68ce-40e6-9b1e-f0e34e8ff9cc,,,26.0,174.0,4729.0,NaN
3,00000981-5e2a-4111-993e-4f1891ad9a53,2023-08-05 01:46:10,view_shop,XVYNT6zfhFWqIg9omwg2AHDjTLx2,00000981-5e2a-4111-993e-4f1891ad9a53,,,61.0,44.0,142.0,NaN
4,00000a7a-ba72-4332-b4a9-7910670aaeb2,2023-07-24 15:03:37,click_bottom_navigation_lab,XFB2SPiGfjbVhvJ3Q3DBsaT3m2B3,00000a7a-ba72-4332-b4a9-7910670aaeb2,,,119.0,545.0,3287.0,NaN


,id,session_id,user_id,language,osname,osversion,versionname,device_id
0,1,4OzYh3seq3VKytpSn5pvQkZNQii1,,ko-KR,iOS,16.0,2.0.0,590E7C79-CBA0-44D8-8BE3-3C9BFABBBC74
1,2,8QXy31PQxbW9qLzq0Y1dhR8Ypm52,1046711,ko-KR,iOS,16.5.1,2.0.3,D5417226-F71B-4A9E-A180-CD072F2AB279
2,3,6bcea65d-9f40-46fc-888c-700fe707483f,1545130,ko,Android,13,2.0.5,6bcea65d-9f40-46fc-888c-700fe707483f
3,4,XVYNT6zfhFWqIg9omwg2AHDjTLx2,1224793,ko,Android,13,2.0.5,a05c1595-3e05-434b-8684-218b528bd725
4,5,XFB2SPiGfjbVhvJ3Q3DBsaT3m2B3,1329450,ko-US,iOS,16.5.1,2.0.5,EAC6C0B3-7CE8-40EA-8A91-9977C0BA5EF3


In [17]:
df_dict["hackle_events"].page_name.unique()

array(['', 'notice', 'home', 'profile', '학교선택', '학년선택', '반선택', '번호인증',
       '성별선택', '아이디입력', '프사설정', 'invite', '이름입력'], dtype=object)

In [23]:
# hackle_events와 hackle_properties의 session_id를 통한 병합
# event_col = [
#     "session_id", "event_id","event_datetime", "event_key"
# ]
# property_col = [
#     "session_id", "user_id", "osname", "osversion","versionname",'device_id'
# ]
# hackle_merged = pd.merge(
#     df_dict["hackle_events"][event_col],
#     df_dict["hackle_properties"][property_col],
#     on="session_id",
#     how="left"
# )

hackle_merged = pd.merge(
    df_dict["hackle_events"],
    df_dict["hackle_properties"],
    on="session_id",
    how="left"
)

# 병합된 데이터 확인
display(hackle_merged.head())
# 병합된 데이터의 크기 확인
print(f"병합된 데이터의 크기: {hackle_merged.shape}")

# 병합되기 전 데이터 크기 확인
print(f"hackle_events 데이터의 크기: {df_dict['hackle_events'].shape}")
print(f"hackle_properties 데이터의 크기: {df_dict['hackle_properties'].shape}")

: 

In [10]:
# session_id 중복 개수 확인
print(df_dict["hackle_events"]["session_id"].duplicated().sum())
print(df_dict["hackle_properties"]["session_id"].duplicated().sum())

11187703
271734


In [13]:
df_dict["hackle_events"].session_id.value_counts().head()

session_id
A40CA2FA-CEB6-4E94-857D-7C229ECC2598    8157
cfa76079-ab4e-4ea1-a7df-38ceaa5ef011    6551
2iO7jwc0YtW0SI8AUjCVoV5gQ1F2            6548
b1b6f602-17ce-4636-9eff-bf0ad5578c5a    5714
6f2a1743-5513-405c-8425-ce11efe267b2    5684
Name: count, dtype: int64

In [19]:
df_dict["hackle_events"][df_dict["hackle_events"].event_key == "complete_signup"]

,event_id,event_datetime,event_key,session_id,id,item_name,page_name,friend_count,votes_count,heart_balance,question_id
20469,0076d12e-cf36-4aa8-8140-dfb1d2eae24b,2023-07-20 19:06:26,complete_signup,EBB7AAFB-6071-458C-A8EA-3B27E8960C71,0076d12e-cf36-4aa8-8140-dfb1d2eae24b,,,0.0,0.0,300.0,NaN
35117,00cabaf0-bd1c-41c8-bcef-02fcceecfefb,2023-07-29 19:47:57,complete_signup,4e9bdccd-18d1-43d4-bcc8-ffcc84ccadba,00cabaf0-bd1c-41c8-bcef-02fcceecfefb,,,0.0,0.0,300.0,NaN
55521,013fb9a3-0b8d-41f5-a690-d58cd398b9a4,2023-07-26 02:23:04,complete_signup,53608491-9A79-4E27-8AC6-B111D0B42A0A,013fb9a3-0b8d-41f5-a690-d58cd398b9a4,,,0.0,0.0,300.0,NaN
63017,016b7031-bcbc-48d3-ad89-c4a5d803b0a8,2023-07-19 13:34:11,complete_signup,679f672f-d5ce-4c6a-bad0-7a8e7070e625,016b7031-bcbc-48d3-ad89-c4a5d803b0a8,,,0.0,0.0,300.0,NaN
73825,01a968a6-52a2-4267-8a27-69e41ee49e87,2023-08-08 10:58:17,complete_signup,005C4D5D-BF73-428A-96D0-160B200497E3,01a968a6-52a2-4267-8a27-69e41ee49e87,,,0.0,0.0,300.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11397793,ff049a95-3817-4ce1-aaf7-782b0d499247,2023-08-01 12:51:29,complete_signup,CF4D1680-AF7A-4AD9-83BA-FAC7A563E444,ff049a95-3817-4ce1-aaf7-782b0d499247,,,0.0,0.0,300.0,NaN
11405709,ff32947d-2cc9-4c7e-8335-0eb8f27387a1,2023-07-24 23:59:41,complete_signup,4ff8b730-88ac-436e-b62a-9d2129bfe15e,ff32947d-2cc9-4c7e-8335-0eb8f27387a1,,,0.0,0.0,300.0,NaN
11412168,ff57b47c-61bc-405d-b77a-8016e02c44ab,2023-07-21 19:19:23,complete_signup,BEC7D7C3-C133-466A-99D4-3165EE5D9B5E,ff57b47c-61bc-405d-b77a-8016e02c44ab,,,0.0,0.0,300.0,NaN
11415006,ff685ed1-0924-4213-bbb3-e51fee6737ef,2023-08-07 22:39:49,complete_signup,715AE207-64D2-4FC3-863F-ADF82B35C4B5,ff685ed1-0924-4213-bbb3-e51fee6737ef,,,0.0,0.0,300.0,NaN


In [18]:
df_dict["hackle_events"][df_dict["hackle_events"].session_id == "A40CA2FA-CEB6-4E94-857D-7C229ECC2598"].sort_values(by="event_datetime")

,event_id,event_datetime,event_key,session_id,id,item_name,page_name,friend_count,votes_count,heart_balance,question_id
2543532,38e3addd-f42e-41bf-a07b-f8ae2d4f9bbf,2023-07-18 00:15:00,launch_app,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,38e3addd-f42e-41bf-a07b-f8ae2d4f9bbf,,,78.0,33.0,422.0,NaN
6788397,97d6e21b-a086-48cf-ab8b-176c2824db48,2023-07-18 00:15:00,$session_start,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,97d6e21b-a086-48cf-ab8b-176c2824db48,,,78.0,33.0,422.0,NaN
7975752,b272590b-4ff5-46d5-ae89-f6af2b7414f5,2023-07-18 00:15:21,launch_app,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,b272590b-4ff5-46d5-ae89-f6af2b7414f5,,,78.0,33.0,422.0,NaN
3490146,4e0b9b14-2492-4fa0-9c0c-16bbfce11b11,2023-07-18 00:15:26,click_question_start,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,4e0b9b14-2492-4fa0-9c0c-16bbfce11b11,,home,78.0,33.0,422.0,NaN
7847307,af93935d-d7a0-4a02-aa63-92dfafd076d1,2023-07-18 00:15:31,skip_question,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,af93935d-d7a0-4a02-aa63-92dfafd076d1,,,78.0,33.0,422.0,4550.0
...,...,...,...,...,...,...,...,...,...,...,...
5446994,79cb74d2-5b42-49e0-aef9-2e4aeeaaf35c,2023-07-26 16:34:22,view_timeline_tap,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,79cb74d2-5b42-49e0-aef9-2e4aeeaaf35c,,,78.0,33.0,706.0,NaN
9883836,dd1d3893-4daf-4390-9812-d357f1b10197,2023-07-26 16:34:23,click_bottom_navigation_questions,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,dd1d3893-4daf-4390-9812-d357f1b10197,,,78.0,33.0,706.0,NaN
3241255,487b9ed1-ef27-4682-81c7-fa19452dfb2c,2023-07-26 16:34:23,view_questions_tap,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,487b9ed1-ef27-4682-81c7-fa19452dfb2c,,,78.0,33.0,706.0,NaN
1037036,17366763-63b7-4529-b27d-0b50c9e93640,2023-07-26 16:34:29,click_question_open,A40CA2FA-CEB6-4E94-857D-7C229ECC2598,17366763-63b7-4529-b27d-0b50c9e93640,,,78.0,33.0,706.0,NaN


In [ ]:
27333186
11441319
525350